In [2]:
from keras.models import Sequential,Model,load_model
from keras.optimizers import SGD
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.merge import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras.backend as K
import cv2 as cv
import matplotlib.pylab as plt
import os
import glob
from sklearn.model_selection import train_test_split
from collections import namedtuple
from skimage.color import rgb2lab

from tensorflow.python.client import device_lib


Using TensorFlow backend.


In [4]:
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17620617316479041064
]


# 데이터 준비

In [5]:
# 이미지 전처리 및 데이터 분리

RESIZED_IMAGE = (200,200)
N_CLASS = 8
Dataset = namedtuple("Dataset", ["X","y"])

In [6]:
def to_tf_format(imgs) :
    return np.stack([img[:,:, np.newaxis] for img in imgs]).astype(np.float32)

def read_dataset_png(roofpath, n_labels, resize_to):
    images, labels = [], []
    
    for i in range(n_labels) :
        # 00, 01 식이니까 02d로 한다. 
        full_path = roofpath + "/" + format(i, "02d") + "/"
#         print(full_path)
        for img_name in glob.glob(full_path+"*.png") :
            
            img = plt.imread(img_name).astype(np.float32)
            img[:,:,0] = img[:,:,0] / 255            
    
            label = np.zeros((n_labels), dtype=np.float)
            label[i] = 1.0
            
            images.append(img.astype(np.float32))
            labels.append(label)
    
    
    return Dataset(X=to_tf_format(images), y=np.matrix(labels).astype(np.float32))

In [7]:
dataset = read_dataset_png("dataset1", N_CLASS, RESIZED_IMAGE)
print(dataset.X.shape)

print(dataset.y[1, :])
print(dataset.X[1, 1, 1, :])

(400, 200, 200, 1, 3)
[[1. 0. 0. 0. 0. 0. 0. 0.]]
[[0.00012303 0.02745098 0.01960784]]


In [8]:
# 훈련용 데이터 // 테스트 데이터 분리

idx_train, idx_test = train_test_split(range(dataset.X.shape[0]), test_size=0.2, random_state=101)

X_train = dataset.X[idx_train, :, :, :, :]
X_test = dataset.X[idx_test, :, :, :, :]
y_train = dataset.y[idx_train, :]
y_test = dataset.y[idx_test, :]

X_train = X_train.reshape(320, 200, 200, 3)
X_test = X_test.reshape(80, 200, 200, 3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(320, 200, 200, 3)
(80, 200, 200, 3)
(320, 8)
(80, 8)


# CNN 훈련 테스트 1 (실패)

In [124]:
# 모델 훈련 
def color_cnn(num_classes):
    input_image = layer.Input(shape=(200,200,3))
    
    # Convolution Layer  1
    conv_1 = layer.Convolution2D(filters=48, kernel_size=(12,12), strides=(4,4),
                                input_shape=(200,200,3), activation='relu')(input_image)
    conv_1 = layer.BatchNormalization()(conv_1)
    conv_1 = layer.MaxPooling2D(pool_size=(3,3), strides=(2,2))(conv_1)
    
    
#     conv_output = layer.Concatenate()([conv_1])
    
    flatten = layer.Flatten()(conv_1)
    
    # FC
    FC_1 = layer.Dense(units=4096, activation='relu')(flatten)
    FC_1 = layer.Dropout(0.6)(FC_1)
    
    output = layer.Dense(units=num_classes, activation = 'softmax')(FC_1)
    
    model = Model(inputs = input_image, outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     model.compile(optimizer=sgd, loss='categorical_crossentropy', metrixs=['accuray'])
    return model
    
    

In [125]:
img_rows, img_cols = 200, 200
num_classes=8
batch_size = 8
nb_epoch = 5

model = color_cnn(num_classes)
# model.fit_generator(training_set, steps_per_epoch=12000, epoxhs=nb_epoch, 
#                    validation_data=test_set, validation_steps=3000, callbacks=callbacks_list)


In [7]:
## GPU가 아닌 CPU 사용
def get_session():
    device_count = {"CPU":1, "GPU":0}
    return tf.Session(config=tf.ConfigProto(device_count = {"CPU":1, "GPU":0}))

In [11]:
def color_net(num_classes):
    # placeholder for input image
    input_image = Input(shape=(200,200,3))
    # ============================================= TOP BRANCH ===================================================
    # first top convolution layer
    top_conv1 = Convolution2D(filters=48,kernel_size=(12,12),strides=(4,4),
                              input_shape=(200,200,3),activation='relu')(input_image)
    top_conv1 = BatchNormalization()(top_conv1)
    top_conv1 = MaxPooling2D(pool_size=(4,4),strides=(2,2))(top_conv1)

    # second top convolution layer
    # split feature map by half
    top_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(top_conv1)
    top_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(top_conv1)

    top_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv2)
    top_top_conv2 = BatchNormalization()(top_top_conv2)
    top_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_top_conv2)

    top_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv2)
    top_bot_conv2 = BatchNormalization()(top_bot_conv2)
    top_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_bot_conv2)

    # third top convolution layer
    # concat 2 feature map
    top_conv3 = Concatenate()([top_top_conv2,top_bot_conv2])
    top_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_conv3)

    # fourth top convolution layer
    # split feature map by half
    top_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(top_conv3)
    top_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(top_conv3)

    top_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)

    # fifth top convolution layer
    top_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_top_conv5 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(top_top_conv5) 

    top_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)
    top_bot_conv5 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(top_bot_conv5)

    # ============================================= TOP BOTTOM ===================================================
    # first bottom convolution layer
    bottom_conv1 = Convolution2D(filters=48,kernel_size=(12,12),strides=(4,4),
                              input_shape=(200,200,3),activation='relu')(input_image)
    bottom_conv1 = BatchNormalization()(bottom_conv1)
    bottom_conv1 = MaxPooling2D(pool_size=(4,4),strides=(2,2))(bottom_conv1)

    # second bottom convolution layer
    # split feature map by half
    bottom_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(bottom_conv1)
    bottom_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(bottom_conv1)

    bottom_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv2)
    bottom_top_conv2 = BatchNormalization()(bottom_top_conv2)
    bottom_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_top_conv2)

    bottom_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv2)
    bottom_bot_conv2 = BatchNormalization()(bottom_bot_conv2)
    bottom_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_bot_conv2)

    # third bottom convolution layer
    # concat 2 feature map
    bottom_conv3 = Concatenate()([bottom_top_conv2,bottom_bot_conv2])
    bottom_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_conv3)

    # fourth bottom convolution layer
    # split feature map by half
    bottom_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(bottom_conv3)
    bottom_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(bottom_conv3)

    bottom_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)

    # fifth bottom convolution layer
    bottom_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_top_conv5 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(bottom_top_conv5) 

    bottom_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)
    bottom_bot_conv5 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(bottom_bot_conv5)

    # ======================================== CONCATENATE TOP AND BOTTOM BRANCH =================================
    conv_output = Concatenate()([top_top_conv5,top_bot_conv5,bottom_top_conv5,bottom_bot_conv5])

    # Flatten
    flatten = Flatten()(conv_output)

    # Fully-connected layer
    FC_1 = Dense(units=4096, activation='relu')(flatten)
    FC_1 = Dropout(0.6)(FC_1)
    FC_2 = Dense(units=4096, activation='relu')(FC_1)
    FC_2 = Dropout(0.6)(FC_2)
    output = Dense(units=num_classes, activation='softmax')(FC_2)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#     model.compile(optimizer=sgd, loss='categorical_crossentropy')
    
    return model

In [12]:
img_rows , img_cols = 200,200
num_classes = 8
batch_size = 5
nb_epoch = 5

model = color_net(num_classes)

filepath = 'color_weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.3,
#         horizontal_flip=True)

# training_set = train_datagen.flow_from_directory(
#             'sample_dataset/',
#             target_size=(img_rows, img_cols),
#             batch_size=batch_size,
#             class_mode='categorical')

model.fit(X_train, y_train, epochs=10, batch_size=200)

# model.save('color_model.h5')

Epoch 1/10
320/320 [==============================] - 12s 36ms/step - loss: 2.6311 - accuracy: 0.1219
Epoch 2/10
320/320 [==============================] - 9s 29ms/step - loss: 2.4467 - accuracy: 0.1594
Epoch 3/10
320/320 [==============================] - 9s 29ms/step - loss: 2.2977 - accuracy: 0.1656
Epoch 4/10
320/320 [==============================] - 9s 30ms/step - loss: 2.1983 - accuracy: 0.1688
Epoch 5/10
320/320 [==============================] - 10s 30ms/step - loss: 2.0658 - accuracy: 0.2000
Epoch 6/10
320/320 [==============================] - 10s 30ms/step - loss: 2.0223 - accuracy: 0.2344
Epoch 7/10
320/320 [==============================] - 9s 30ms/step - loss: 1.9415 - accuracy: 0.2344
Epoch 8/10
320/320 [==============================] - 10s 30ms/step - loss: 1.8382 - accuracy: 0.3000
Epoch 9/10
320/320 [==============================] - 9s 30ms/step - loss: 1.7553 - accuracy: 0.3375
Epoch 10/10
320/320 [==============================] - 10s 30ms/step - loss: 1.7318 - a

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 48, 48, 48)   20784       input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 48, 48, 48)   20784       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 48, 48, 48)   192         conv2d_5[0][0]                   
____________________________________________________________________________________________

## 정확도

In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print("정확도 : ", test_acc)

정확도 :  0.125


## 예측

In [20]:
img = plt.imread('dataset/04/100.png').reshape(1,200,200,3)

prediction = model.predict(img)

print(prediction)

np.argmax(prediction)

[[0.1271185  0.13348615 0.1208934  0.12126196 0.13801192 0.1308624
  0.11689004 0.11147561]]


4